<h1 id="dataset" style="color:#FFFFFF; background:#205375; border:1.5px dotted;"> 
    <center>kaggle_environments
        <a class="anchor-link" href="#dataset" target="_self"></a>
    </center>
</h1>

In [ ]:
%%capture
# install the latest version of kaggle_environments
!pip install --upgrade kaggle_environments

In [ ]:
from kaggle_environments import make
env = make("kore_fleets", debug=True)
print(env.name, env.version)

<h1 id="dataset" style="color:#FFFFFF; background:#A0BCC2; border:1.5px dotted;"> 
    <center>miner.py
        <a class="anchor-link" href="#dataset" target="_self"></a>
    </center>
</h1>

In [ ]:
%%writefile miner.py

import pandas as pd
from random import random, sample, randint
from kaggle_environments import utils
from kaggle_environments.helpers import Point, Direction
from kaggle_environments.envs.kore_fleets.helpers import Board, ShipyardAction
from PIL import Image as im
import numpy as np
import matplotlib.pyplot as plt 

def get_kore(obs, config):
    GRID_SIZE = config.size
    kore_grid_ = np.flip(np.array(obs["kore"], dtype=np.float32).reshape(GRID_SIZE, GRID_SIZE), axis=0)
    return kore_grid_

def save_image(array):    
    #array = np.where(array >10, 50, array)
    
    # in the final turns the kore increases but I normalize the data
    array = array / np.sqrt(np.sum(array**2))
    array=array*255
    array=np.uint16(array)
    
    #put data 5,5 or data 15,15 in 255
    array[5,5]=255
    #array[15,15]=255
    array = (array * 255).astype(np.uint16)
   
    data = im.fromarray(array)
    data.save('separated100.png')    
    
def generate_image(array):
    # in the final turns the kore increases but I normalize the data
    array = array / np.sqrt(np.sum(array**2))
    array=array*255
    array=np.uint8(array)  
    
    #array = np.where(array >30, 200, array)    
    array[5,5]=255
    #array[15,15]=255
    print(array)  
    
    plt.imshow(array)
    plt.show() 
    
def miner_agent(obs, config):
    board = Board(obs, config)
    me = board.current_player
    kore_left = me.kore
    shipyards = me.shipyards
    
    spawn_cost = board.configuration.spawn_cost
    # randomize shipyard order
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    kore_grid = get_kore(obs, config)
    #*************************************************************
    #*********************IMPORTANT*******************************
    #*************************************************************
    # GENERATE 1,50,100 AND 200 TURNS
    #you can change the turns
    # in the final turns the kore increases but I normalize the data
    
    if turn==1:
        generate_image(kore_grid)
        save_image(kore_grid)
        
    if turn==50:
        generate_image(kore_grid)
        
    if turn==100:
        generate_image(kore_grid)
        
    if turn==200:
        generate_image(kore_grid)       
        
        
    shipyards = sample(shipyards, len(shipyards))
    period =7
    
    for shipyard in shipyards:
        action = None
        #I only put a flight plan, to test
        if turn % period == 2:
            if(shipyard.ship_count >= 22):
                action = ShipyardAction.launch_fleet_with_flight_plan(22, "S3E8N3W")
                shipyard.next_action = action
           
            elif kore_left > board.configuration.spawn_cost * shipyard.max_spawn:
                kore_left -= board.configuration.spawn_cost
                if kore_left >= spawn_cost:
                    shipyard.next_action = ShipyardAction.spawn_ships(min(shipyard.max_spawn, int(kore_left/spawn_cost))) 
    return me.next_actions

<h1 id="dataset" style="color:#FFFFFF; background:#A0BCC2; border:1.5px dotted;"> 
    <center>GENERATE IMAGES IN  1,50,100 AND 200 TURNS
        <a class="anchor-link" href="#dataset" target="_self"></a>
    </center>
    <center style="color:#000000;" >IF I HELP YOU, GIVE ME A VOTE
        <a class="anchor-link" href="#dataset" target="_self"></a>
    </center>
</h1>

In [ ]:
env.run(["/kaggle/working/miner.py", "do_nothing"])
env.render(mode="ipython", width=1000, height=800)